# Linear Regression Exercises - Solutions

This notebook contains detailed solutions to all exercises from the Linear Models notebook, with explanations and business interpretations.

## Setup

First, let's import all necessary libraries and recreate the models from the main notebook.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.dummy import DummyRegressor
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
np.random.seed(42)

In [ ]:
# Recreate the banking dataset model from the main notebook
banking_url = "https://raw.githubusercontent.com/umatter/EDFB/main/data/banking.csv"
dataset = pd.read_csv(banking_url)

# Prepare the data exactly as in the main notebook
df_banking = dataset.copy()
df_banking['was_previously_contacted'] = (df_banking['pdays'] != 999).astype(int)
df_banking['pdays_clean'] = df_banking['pdays'].replace(999, np.nan)
df_banking['pdays_clean'] = df_banking['pdays_clean'].fillna(df_banking['pdays_clean'].median())

feature_cols_cat = ['marital', 'education', 'housing', 'loan', 'contact', 'poutcome']
feature_cols_num = ['age', 'previous', 'pdays_clean', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'was_previously_contacted']
X_df = pd.get_dummies(df_banking[feature_cols_cat + feature_cols_num], drop_first=True)
X = X_df.values
y = np.log1p(df_banking['duration']).values.reshape(-1,1)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)
y_test_predicted = model.predict(X_test)

print(f"Banking model trained successfully!")
print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Number of features: {len(X_df.columns)}")

---

## Exercise 1: Understanding Model Coefficients

**Task:** Interpret the coefficients from the banking dataset model and understand their business meaning.

In [ ]:
# Solution for Exercise 1

# Create a DataFrame with feature names and coefficients
coefficients_df = pd.DataFrame({
    'Feature': X_df.columns,
    'Coefficient': model.coef_[0],
    'Abs_Coefficient': np.abs(model.coef_[0])
})

# Sort by absolute value to see most important features
coefficients_df = coefficients_df.sort_values('Abs_Coefficient', ascending=False)

print("=== MODEL COEFFICIENTS ANALYSIS ===")
print(f"Intercept: {model.intercept_[0]:.4f}")
print("\nAll coefficients (sorted by absolute magnitude):")
print(coefficients_df.to_string(index=False))

print("\n=== TOP 5 POSITIVE COEFFICIENTS ===")
top_positive = coefficients_df[coefficients_df['Coefficient'] > 0].head(5)
for _, row in top_positive.iterrows():
    print(f"{row['Feature']}: {row['Coefficient']:.4f}")

print("\n=== TOP 5 NEGATIVE COEFFICIENTS ===")
top_negative = coefficients_df[coefficients_df['Coefficient'] < 0].head(5)
for _, row in top_negative.iterrows():
    print(f"{row['Feature']}: {row['Coefficient']:.4f}")

In [ ]:
# Visualize the most important coefficients
plt.figure(figsize=(12, 8))
top_features = coefficients_df.head(15)  # Top 15 most important features

colors = ['red' if coef < 0 else 'blue' for coef in top_features['Coefficient']]
plt.barh(range(len(top_features)), top_features['Coefficient'], color=colors, alpha=0.7)
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Coefficient Value')
plt.title('Top 15 Most Important Features (by absolute coefficient value)')
plt.axvline(x=0, color='black', linestyle='--', alpha=0.5)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Business Interpretation of Coefficients

**Key Insights:**

1. **Positive coefficients** increase call duration (log scale):
   - Features that lead to longer, more engaged conversations
   - May indicate higher customer interest or more complex discussions

2. **Negative coefficients** decrease call duration:
   - Features associated with quick rejections or disinterest
   - May indicate customers who are not good targets

3. **Magnitude matters**: Larger absolute coefficients have stronger impact on call duration

**Business Applications:**
- Use positive predictors to identify high-engagement prospects
- Avoid or deprioritize customers with strong negative predictors
- Tailor call scripts based on customer characteristics

---

## Exercise 2: Residual Analysis

**Task:** Perform a comprehensive residual analysis to check model assumptions.

In [ ]:
# Solution for Exercise 2

# Calculate residuals for both training and test sets
y_train_predicted = model.predict(X_train)
residuals_train = y_train.ravel() - y_train_predicted.ravel()
residuals_test = y_test.ravel() - y_test_predicted.ravel()

print("=== RESIDUAL ANALYSIS ===")
print(f"Training residuals - Mean: {residuals_train.mean():.6f}, Std: {residuals_train.std():.4f}")
print(f"Test residuals - Mean: {residuals_test.mean():.6f}, Std: {residuals_test.std():.4f}")

# Create comprehensive residual plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Residuals vs Fitted Values (Training)
axes[0,0].scatter(y_train_predicted.ravel(), residuals_train, alpha=0.6)
axes[0,0].axhline(y=0, color='red', linestyle='--')
axes[0,0].set_xlabel('Fitted Values')
axes[0,0].set_ylabel('Residuals')
axes[0,0].set_title('Residuals vs Fitted (Training)')
axes[0,0].grid(True, alpha=0.3)

# 2. Residuals vs Fitted Values (Test)
axes[0,1].scatter(y_test_predicted.ravel(), residuals_test, alpha=0.6, color='orange')
axes[0,1].axhline(y=0, color='red', linestyle='--')
axes[0,1].set_xlabel('Fitted Values')
axes[0,1].set_ylabel('Residuals')
axes[0,1].set_title('Residuals vs Fitted (Test)')
axes[0,1].grid(True, alpha=0.3)

# 3. Histogram of residuals (Test)
axes[0,2].hist(residuals_test, bins=30, alpha=0.7, color='green', edgecolor='black')
axes[0,2].set_xlabel('Residuals')
axes[0,2].set_ylabel('Frequency')
axes[0,2].set_title('Distribution of Residuals (Test)')
axes[0,2].grid(True, alpha=0.3)

# 4. Q-Q Plot for normality check
stats.probplot(residuals_test, dist="norm", plot=axes[1,0])
axes[1,0].set_title('Q-Q Plot (Test Residuals)')
axes[1,0].grid(True, alpha=0.3)

# 5. Residuals over time (index)
axes[1,1].plot(residuals_test, alpha=0.7)
axes[1,1].axhline(y=0, color='red', linestyle='--')
axes[1,1].set_xlabel('Observation Index')
axes[1,1].set_ylabel('Residuals')
axes[1,1].set_title('Residuals Over Index (Test)')
axes[1,1].grid(True, alpha=0.3)

# 6. Scale-Location plot (sqrt of absolute residuals vs fitted)
sqrt_abs_resid = np.sqrt(np.abs(residuals_test))
axes[1,2].scatter(y_test_predicted.ravel(), sqrt_abs_resid, alpha=0.6, color='purple')
axes[1,2].set_xlabel('Fitted Values')
axes[1,2].set_ylabel('√|Residuals|')
axes[1,2].set_title('Scale-Location Plot (Test)')
axes[1,2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Statistical tests for assumptions
from scipy.stats import shapiro, jarque_bera

print("=== STATISTICAL TESTS FOR ASSUMPTIONS ===")

# Test for normality of residuals
shapiro_stat, shapiro_p = shapiro(residuals_test[:5000])  # Shapiro-Wilk (limited sample)
jb_stat, jb_p = jarque_bera(residuals_test)  # Jarque-Bera test

print(f"\n1. NORMALITY TESTS:")
print(f"   Shapiro-Wilk test: statistic={shapiro_stat:.4f}, p-value={shapiro_p:.6f}")
print(f"   Jarque-Bera test: statistic={jb_stat:.4f}, p-value={jb_p:.6f}")
print(f"   Interpretation: {'Residuals appear normal' if jb_p > 0.05 else 'Residuals deviate from normality'}")

# Check for heteroscedasticity (Breusch-Pagan test would be ideal, but we'll use correlation)
fitted_values = y_test_predicted.ravel()
abs_residuals = np.abs(residuals_test)
hetero_corr = np.corrcoef(fitted_values, abs_residuals)[0,1]

print(f"\n2. HOMOSCEDASTICITY CHECK:")
print(f"   Correlation between fitted values and |residuals|: {hetero_corr:.4f}")
print(f"   Interpretation: {'Homoscedasticity likely' if abs(hetero_corr) < 0.1 else 'Potential heteroscedasticity'}")

# Summary statistics
print(f"\n3. RESIDUAL SUMMARY:")
print(f"   Mean: {residuals_test.mean():.6f} (should be ≈ 0)")
print(f"   Standard deviation: {residuals_test.std():.4f}")
print(f"   Skewness: {stats.skew(residuals_test):.4f} (should be ≈ 0)")
print(f"   Kurtosis: {stats.kurtosis(residuals_test):.4f} (should be ≈ 0)")

### Interpretation of Residual Analysis

**What to look for:**

1. **Residuals vs Fitted**: Should show random scatter around zero
   - Patterns indicate non-linearity or heteroscedasticity
   - Funnel shapes suggest changing variance

2. **Q-Q Plot**: Points should follow the diagonal line
   - Deviations indicate non-normal residuals
   - Heavy tails or skewness are problematic

3. **Histogram**: Should approximate normal distribution
   - Skewness or multiple peaks are concerning

4. **Scale-Location**: Should show constant spread
   - Increasing/decreasing trends indicate heteroscedasticity

**Business Implications:**
- Assumption violations may lead to unreliable predictions
- Consider data transformations or different models if violations are severe

---

## Exercise 3: Feature Engineering

**Task:** Create new features and see if they improve model performance.

In [ ]:
# Solution for Exercise 3

print("=== FEATURE ENGINEERING EXPERIMENT ===")

# Start with numerical features only for polynomial expansion
numerical_features = ['age', 'previous', 'pdays_clean', 'emp_var_rate', 'cons_price_idx', 
                     'cons_conf_idx', 'euribor3m', 'nr_employed', 'was_previously_contacted']

# Get numerical data
X_numerical = df_banking[numerical_features].fillna(0)

# Create polynomial features (degree 2 for interaction terms and squares)
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
X_poly = poly.fit_transform(X_numerical)

# Get feature names
poly_feature_names = poly.get_feature_names_out(numerical_features)

print(f"Original features: {len(numerical_features)}")
print(f"Polynomial features: {len(poly_feature_names)}")
print(f"\nFirst 10 polynomial features:")
for i, name in enumerate(poly_feature_names[:10]):
    print(f"  {i+1}. {name}")

# Split the polynomial data
X_poly_train, X_poly_test, y_poly_train, y_poly_test = train_test_split(
    X_poly, y, test_size=0.2, random_state=0
)

# Scale the features (important for polynomial features)
scaler = StandardScaler()
X_poly_train_scaled = scaler.fit_transform(X_poly_train)
X_poly_test_scaled = scaler.transform(X_poly_test)

# Train models
# 1. Original model (for comparison)
original_model = LinearRegression()
original_model.fit(X_train, y_train)
y_pred_original = original_model.predict(X_test)

# 2. Polynomial model
poly_model = LinearRegression()
poly_model.fit(X_poly_train_scaled, y_poly_train)
y_pred_poly = poly_model.predict(X_poly_test_scaled)

# 3. Ridge regression (to handle potential overfitting)
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_poly_train_scaled, y_poly_train)
y_pred_ridge = ridge_model.predict(X_poly_test_scaled)

# Compare performance
models_comparison = {
    'Original Linear': {
        'R²': r2_score(y_test, y_pred_original),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_original)),
        'Features': X_train.shape[1]
    },
    'Polynomial': {
        'R²': r2_score(y_poly_test, y_pred_poly),
        'RMSE': np.sqrt(mean_squared_error(y_poly_test, y_pred_poly)),
        'Features': X_poly_train_scaled.shape[1]
    },
    'Ridge (Polynomial)': {
        'R²': r2_score(y_poly_test, y_pred_ridge),
        'RMSE': np.sqrt(mean_squared_error(y_poly_test, y_pred_ridge)),
        'Features': X_poly_train_scaled.shape[1]
    }
}

print("\n=== MODEL COMPARISON ===")
comparison_df = pd.DataFrame(models_comparison).T
print(comparison_df.round(4))

In [ ]:
# Visualize the comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# R² comparison
r2_scores = [models_comparison[model]['R²'] for model in models_comparison.keys()]
model_names = list(models_comparison.keys())
axes[0].bar(model_names, r2_scores, color=['blue', 'orange', 'green'], alpha=0.7)
axes[0].set_ylabel('R² Score')
axes[0].set_title('R² Comparison')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# RMSE comparison
rmse_scores = [models_comparison[model]['RMSE'] for model in models_comparison.keys()]
axes[1].bar(model_names, rmse_scores, color=['blue', 'orange', 'green'], alpha=0.7)
axes[1].set_ylabel('RMSE')
axes[1].set_title('RMSE Comparison (Lower is Better)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

# Feature count comparison
feature_counts = [models_comparison[model]['Features'] for model in models_comparison.keys()]
axes[2].bar(model_names, feature_counts, color=['blue', 'orange', 'green'], alpha=0.7)
axes[2].set_ylabel('Number of Features')
axes[2].set_title('Model Complexity')
axes[2].tick_params(axis='x', rotation=45)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show some important polynomial features
if len(poly_model.coef_[0]) > 0:
    poly_coef_df = pd.DataFrame({
        'Feature': poly_feature_names,
        'Coefficient': poly_model.coef_[0],
        'Abs_Coefficient': np.abs(poly_model.coef_[0])
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print("\n=== TOP 10 POLYNOMIAL FEATURES ===")
    print(poly_coef_df.head(10)[['Feature', 'Coefficient']].to_string(index=False))

### Feature Engineering Insights

**Key Findings:**

1. **Polynomial Features**: Adding interaction terms and squared features
   - Captures non-linear relationships
   - May improve fit but risk overfitting

2. **Regularization**: Ridge regression helps control overfitting
   - Balances bias-variance tradeoff
   - Often performs better on new data

3. **Complexity vs Performance**: More features ≠ better performance
   - Diminishing returns from additional complexity
   - Simple models often generalize better

**Business Implications:**
- Feature engineering can capture complex customer behaviors
- Balance model complexity with interpretability needs
- Always validate on out-of-sample data

---

## Exercise 4: Cross-Validation

**Task:** Use cross-validation to get a more robust estimate of model performance.

In [ ]:
# Solution for Exercise 4

print("=== CROSS-VALIDATION ANALYSIS ===")

# Prepare models for cross-validation
models_cv = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Baseline (Mean)': DummyRegressor(strategy='mean')
}

# Perform 5-fold cross-validation
cv_results = {}
n_folds = 5

for name, model in models_cv.items():
    # Use original features for fair comparison
    scores = cross_val_score(model, X, y.ravel(), cv=n_folds, scoring='r2')
    cv_results[name] = {
        'scores': scores,
        'mean': scores.mean(),
        'std': scores.std(),
        'min': scores.min(),
        'max': scores.max()
    }

# Display results
print(f"\n{n_folds}-Fold Cross-Validation Results (R² scores):")
print("=" * 60)

for name, results in cv_results.items():
    print(f"\n{name}:")
    print(f"  Mean R²: {results['mean']:.4f} (±{results['std']:.4f})")
    print(f"  Range: [{results['min']:.4f}, {results['max']:.4f}]")
    print(f"  Individual folds: {[f'{score:.4f}' for score in results['scores']]}")

# Statistical significance test
lr_scores = cv_results['Linear Regression']['scores']
ridge_scores = cv_results['Ridge Regression']['scores']
baseline_scores = cv_results['Baseline (Mean)']['scores']

# Paired t-test between Linear Regression and Ridge
t_stat_lr_ridge, p_val_lr_ridge = stats.ttest_rel(lr_scores, ridge_scores)
t_stat_lr_baseline, p_val_lr_baseline = stats.ttest_rel(lr_scores, baseline_scores)

print(f"\n=== STATISTICAL SIGNIFICANCE TESTS ===")
print(f"Linear vs Ridge: t-statistic={t_stat_lr_ridge:.4f}, p-value={p_val_lr_ridge:.4f}")
print(f"Linear vs Baseline: t-statistic={t_stat_lr_baseline:.4f}, p-value={p_val_lr_baseline:.4f}")
print(f"\nSignificance level: α = 0.05")
print(f"Linear vs Ridge: {'Significantly different' if p_val_lr_ridge < 0.05 else 'No significant difference'}")
print(f"Linear vs Baseline: {'Significantly better' if p_val_lr_baseline < 0.05 else 'No significant improvement'}")

In [ ]:
# Visualize cross-validation results
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Box plot of CV scores
cv_data = [cv_results[name]['scores'] for name in cv_results.keys()]
cv_labels = list(cv_results.keys())

box_plot = axes[0].boxplot(cv_data, labels=cv_labels, patch_artist=True)
colors = ['lightblue', 'lightgreen', 'lightcoral']
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)

axes[0].set_ylabel('R² Score')
axes[0].set_title('Cross-Validation Score Distribution')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Mean scores with error bars
means = [cv_results[name]['mean'] for name in cv_results.keys()]
stds = [cv_results[name]['std'] for name in cv_results.keys()]

bars = axes[1].bar(cv_labels, means, yerr=stds, capsize=5, 
                   color=colors, alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Mean R² Score')
axes[1].set_title('Mean CV Performance with Standard Deviation')
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar, mean, std in zip(bars, means, stds):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height + std + 0.01,
                f'{mean:.3f}±{std:.3f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

# Model stability analysis
print("\n=== MODEL STABILITY ANALYSIS ===")
for name, results in cv_results.items():
    cv_coefficient = results['std'] / results['mean'] if results['mean'] != 0 else float('inf')
    stability = "High" if cv_coefficient < 0.1 else "Medium" if cv_coefficient < 0.2 else "Low"
    print(f"{name}: CV = {cv_coefficient:.3f} ({stability} stability)")

### Cross-Validation Insights

**Why Cross-Validation Matters:**

1. **Robust Performance Estimate**: Single train-test split can be misleading
2. **Model Stability**: Shows how consistent performance is across different data samples
3. **Overfitting Detection**: Large gap between training and CV performance indicates overfitting
4. **Model Selection**: Compare models fairly using same data splits

**Business Implications:**
- More reliable estimate of real-world performance
- Helps choose between competing models
- Identifies models that may fail in production
- Builds confidence in model deployment decisions

---

## Exercise 5: Synthetic Data Generation

**Task:** Create your own synthetic dataset with known relationships and test your model.

In [ ]:
# Solution for Exercise 5

print("=== SYNTHETIC DATA GENERATION EXPERIMENT ===")

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic dataset
n_samples = 500
true_coefficients = [2.0, 3.0, -1.5]  # Known true relationship
true_intercept = 1.0

# Generate features
X_synthetic = np.random.randn(n_samples, 3)  # 3 features, standard normal
feature_names = ['x1', 'x2', 'x3']

# Create true relationship: y = 1.0 + 2.0*x1 + 3.0*x2 - 1.5*x3 + noise
def generate_target(X, coefficients, intercept, noise_level=0.5):
    """Generate target variable with known linear relationship"""
    y_true = intercept + np.dot(X, coefficients)
    noise = np.random.normal(0, noise_level, len(y_true))
    return y_true + noise, y_true

# Test different noise levels
noise_levels = [0.1, 0.5, 1.0, 2.0]
results_by_noise = {}

for noise_level in noise_levels:
    print(f"\n--- Testing with noise level: {noise_level} ---")
    
    # Generate data with current noise level
    y_synthetic, y_true = generate_target(X_synthetic, true_coefficients, true_intercept, noise_level)
    
    # Add some outliers (5% of data)
    n_outliers = int(0.05 * n_samples)
    outlier_indices = np.random.choice(n_samples, n_outliers, replace=False)
    y_synthetic[outlier_indices] += np.random.normal(0, 5*noise_level, n_outliers)
    
    # Split data
    X_train_syn, X_test_syn, y_train_syn, y_test_syn = train_test_split(
        X_synthetic, y_synthetic, test_size=0.2, random_state=42
    )
    
    # Train model
    model_syn = LinearRegression()
    model_syn.fit(X_train_syn, y_train_syn)
    
    # Evaluate
    y_pred_syn = model_syn.predict(X_test_syn)
    r2_syn = r2_score(y_test_syn, y_pred_syn)
    rmse_syn = np.sqrt(mean_squared_error(y_test_syn, y_pred_syn))
    
    # Compare recovered coefficients with true coefficients
    recovered_coef = model_syn.coef_
    recovered_intercept = model_syn.intercept_
    
    coef_errors = np.abs(recovered_coef - true_coefficients)
    intercept_error = abs(recovered_intercept - true_intercept)
    
    results_by_noise[noise_level] = {
        'r2': r2_syn,
        'rmse': rmse_syn,
        'coef_errors': coef_errors,
        'intercept_error': intercept_error,
        'recovered_coef': recovered_coef,
        'recovered_intercept': recovered_intercept
    }
    
    print(f"R²: {r2_syn:.4f}, RMSE: {rmse_syn:.4f}")
    print(f"True coefficients: {true_coefficients}")
    print(f"Recovered coefficients: {[f'{c:.3f}' for c in recovered_coef]}")
    print(f"Coefficient errors: {[f'{e:.3f}' for e in coef_errors]}")
    print(f"True intercept: {true_intercept:.3f}, Recovered: {recovered_intercept:.3f}")

In [ ]:
# Visualize the effect of noise on model performance
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Extract results for plotting
noise_vals = list(results_by_noise.keys())
r2_vals = [results_by_noise[n]['r2'] for n in noise_vals]
rmse_vals = [results_by_noise[n]['rmse'] for n in noise_vals]

# R² vs Noise Level
axes[0,0].plot(noise_vals, r2_vals, 'bo-', linewidth=2, markersize=8)
axes[0,0].set_xlabel('Noise Level')
axes[0,0].set_ylabel('R² Score')
axes[0,0].set_title('Model Performance vs Noise Level')
axes[0,0].grid(True, alpha=0.3)

# RMSE vs Noise Level
axes[0,1].plot(noise_vals, rmse_vals, 'ro-', linewidth=2, markersize=8)
axes[0,1].set_xlabel('Noise Level')
axes[0,1].set_ylabel('RMSE')
axes[0,1].set_title('RMSE vs Noise Level')
axes[0,1].grid(True, alpha=0.3)

# Coefficient recovery accuracy
coef_errors_by_feature = np.array([results_by_noise[n]['coef_errors'] for n in noise_vals])
for i, feature in enumerate(feature_names):
    axes[1,0].plot(noise_vals, coef_errors_by_feature[:, i], 'o-', 
                   label=f'{feature} (true: {true_coefficients[i]})', linewidth=2, markersize=6)
axes[1,0].set_xlabel('Noise Level')
axes[1,0].set_ylabel('Absolute Coefficient Error')
axes[1,0].set_title('Coefficient Recovery Accuracy')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Scatter plot for highest noise case
highest_noise = max(noise_vals)
y_synthetic_final, _ = generate_target(X_synthetic, true_coefficients, true_intercept, highest_noise)
model_final = LinearRegression().fit(X_synthetic, y_synthetic_final)
y_pred_final = model_final.predict(X_synthetic)

axes[1,1].scatter(y_synthetic_final, y_pred_final, alpha=0.6)
axes[1,1].plot([y_synthetic_final.min(), y_synthetic_final.max()], 
               [y_synthetic_final.min(), y_synthetic_final.max()], 'r--', linewidth=2)
axes[1,1].set_xlabel('True Values')
axes[1,1].set_ylabel('Predicted Values')
axes[1,1].set_title(f'Predictions vs True (Noise={highest_noise})')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
summary_df = pd.DataFrame({
    'Noise_Level': noise_vals,
    'R²': [f"{results_by_noise[n]['r2']:.4f}" for n in noise_vals],
    'RMSE': [f"{results_by_noise[n]['rmse']:.4f}" for n in noise_vals],
    'Mean_Coef_Error': [f"{np.mean(results_by_noise[n]['coef_errors']):.4f}" for n in noise_vals],
    'Intercept_Error': [f"{results_by_noise[n]['intercept_error']:.4f}" for n in noise_vals]
})

print("\n=== SUMMARY TABLE ===")
print(summary_df.to_string(index=False))

### Synthetic Data Insights

**Key Learnings:**

1. **Noise Impact**: Higher noise levels reduce model performance and coefficient accuracy
2. **Coefficient Recovery**: Linear regression can recover true relationships when assumptions are met
3. **Outlier Effects**: Even small percentages of outliers can significantly impact results
4. **Sample Size**: Larger samples generally lead to better coefficient recovery

**Business Applications:**
- Understand how data quality affects model reliability
- Set realistic expectations for model performance
- Identify when additional data collection might be needed
- Test model robustness before deployment

---

## Exercise 6: Time Series Forecasting Analysis

**Task:** Analyze the Bitcoin forecasting model more deeply.

In [ ]:
# Solution for Exercise 6
# First, let's recreate the Bitcoin data and model from the main notebook

print("=== BITCOIN FORECASTING ANALYSIS ===")

# Load Bitcoin data
try:
    btc_url = "https://raw.githubusercontent.com/umatter/EDFB/main/data/data_BTC.csv"
    data_btc = pd.read_csv(btc_url)
except:
    print("Using fallback synthetic Bitcoin data for demonstration...")
    # Create synthetic Bitcoin-like data for demonstration
    dates = pd.date_range('2020-01-01', periods=1000, freq='D')
    np.random.seed(42)
    prices = 10000 * np.exp(np.cumsum(np.random.normal(0.001, 0.02, 1000)))
    data_btc = pd.DataFrame({
        'Date': dates,
        'BTC-USD.Close': prices
    })

# Create lagged features function
def create_lagged_features_btc(data, lag):
    df = data.copy()
    df['ret'] = np.log(df['BTC-USD.Close']).diff()
    for i in range(1, lag+1):
        df[f'lag_ret_{i}'] = df['ret'].shift(i)
    df = df.dropna().reset_index(drop=True)
    return df

# Test different lag lengths
lag_lengths = [1, 3, 5, 10]
lag_results = {}

for lag in lag_lengths:
    print(f"\n--- Testing lag length: {lag} ---")
    
    # Create features
    data_lagged = create_lagged_features_btc(data_btc, lag)
    
    # Prepare features and target
    feature_cols = [f'lag_ret_{i}' for i in range(1, lag+1)]
    X_btc = data_lagged[feature_cols]
    y_btc = data_lagged['ret']
    
    # Chronological split (important for time series!)
    split_idx = int(len(data_lagged) * 0.8)
    X_train_btc = X_btc.iloc[:split_idx]
    X_test_btc = X_btc.iloc[split_idx:]
    y_train_btc = y_btc.iloc[:split_idx]
    y_test_btc = y_btc.iloc[split_idx:]
    
    # Train model
    model_btc = LinearRegression()
    model_btc.fit(X_train_btc, y_train_btc)
    
    # Predictions
    y_pred_btc = model_btc.predict(X_test_btc)
    
    # Performance metrics
    r2_btc = r2_score(y_test_btc, y_pred_btc)
    rmse_btc = np.sqrt(mean_squared_error(y_test_btc, y_pred_btc))
    
    # Directional accuracy
    actual_direction = np.sign(y_test_btc)
    predicted_direction = np.sign(y_pred_btc)
    directional_accuracy = np.mean(actual_direction == predicted_direction)
    
    # Naive baseline (predict last return)
    naive_pred = X_test_btc.iloc[:, 0]  # lag_ret_1
    rmse_naive = np.sqrt(mean_squared_error(y_test_btc, naive_pred))
    
    lag_results[lag] = {
        'r2': r2_btc,
        'rmse': rmse_btc,
        'rmse_naive': rmse_naive,
        'directional_accuracy': directional_accuracy,
        'y_test': y_test_btc,
        'y_pred': y_pred_btc,
        'model': model_btc
    }
    
    print(f"R²: {r2_btc:.4f}")
    print(f"RMSE (model): {rmse_btc:.6f}")
    print(f"RMSE (naive): {rmse_naive:.6f}")
    print(f"RMSE ratio: {rmse_btc/rmse_naive:.3f}")
    print(f"Directional accuracy: {directional_accuracy:.3f}")

In [ ]:
# Visualize lag length comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Performance metrics by lag length
lags = list(lag_results.keys())
r2_scores = [lag_results[lag]['r2'] for lag in lags]
rmse_ratios = [lag_results[lag]['rmse'] / lag_results[lag]['rmse_naive'] for lag in lags]
dir_accuracies = [lag_results[lag]['directional_accuracy'] for lag in lags]

# R² by lag length
axes[0,0].plot(lags, r2_scores, 'bo-', linewidth=2, markersize=8)
axes[0,0].set_xlabel('Lag Length')
axes[0,0].set_ylabel('R² Score')
axes[0,0].set_title('R² vs Lag Length')
axes[0,0].grid(True, alpha=0.3)
axes[0,0].axhline(y=0, color='red', linestyle='--', alpha=0.5)

# RMSE ratio (model/naive)
axes[0,1].plot(lags, rmse_ratios, 'ro-', linewidth=2, markersize=8)
axes[0,1].set_xlabel('Lag Length')
axes[0,1].set_ylabel('RMSE Ratio (Model/Naive)')
axes[0,1].set_title('RMSE Improvement vs Lag Length')
axes[0,1].axhline(y=1, color='black', linestyle='--', alpha=0.5, label='No improvement')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Directional accuracy
axes[1,0].plot(lags, dir_accuracies, 'go-', linewidth=2, markersize=8)
axes[1,0].set_xlabel('Lag Length')
axes[1,0].set_ylabel('Directional Accuracy')
axes[1,0].set_title('Directional Accuracy vs Lag Length')
axes[1,0].axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Random guess')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Time series plot for best performing lag
best_lag = max(lag_results.keys(), key=lambda x: lag_results[x]['directional_accuracy'])
best_results = lag_results[best_lag]

# Plot actual vs predicted returns
plot_length = min(100, len(best_results['y_test']))  # Plot last 100 observations
x_axis = range(plot_length)
axes[1,1].plot(x_axis, best_results['y_test'].iloc[-plot_length:], 
               label='Actual Returns', alpha=0.7, linewidth=1.5)
axes[1,1].plot(x_axis, best_results['y_pred'][-plot_length:], 
               label='Predicted Returns', alpha=0.8, linewidth=1.5)
axes[1,1].set_xlabel('Time')
axes[1,1].set_ylabel('Returns')
axes[1,1].set_title(f'Actual vs Predicted Returns (Lag={best_lag})')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nBest performing lag length: {best_lag} (highest directional accuracy: {best_results['directional_accuracy']:.3f})")

In [ ]:
# Trading strategy analysis
print("\n=== TRADING STRATEGY ANALYSIS ===")

# Use best performing model for trading simulation
best_results = lag_results[best_lag]
y_test_btc = best_results['y_test']
y_pred_btc = best_results['y_pred']

# Create trading signals
# Strategy: Go long if predicted return > threshold, short if < -threshold, hold otherwise
thresholds = [0.0, 0.001, 0.002, 0.005]
strategy_results = {}

for threshold in thresholds:
    # Generate signals
    signals = np.where(y_pred_btc > threshold, 1,  # Long position
                      np.where(y_pred_btc < -threshold, -1, 0))  # Short position, Hold
    
    # Calculate strategy returns
    strategy_returns = signals * y_test_btc.values
    
    # Performance metrics
    total_return = np.sum(strategy_returns)
    cumulative_returns = np.cumsum(strategy_returns)
    sharpe_ratio = np.mean(strategy_returns) / np.std(strategy_returns) if np.std(strategy_returns) > 0 else 0
    max_drawdown = np.max(np.maximum.accumulate(cumulative_returns) - cumulative_returns)
    
    # Hit rate (percentage of profitable trades)
    profitable_trades = np.sum(strategy_returns > 0)
    total_trades = np.sum(signals != 0)
    hit_rate = profitable_trades / total_trades if total_trades > 0 else 0
    
    strategy_results[threshold] = {
        'total_return': total_return,
        'cumulative_returns': cumulative_returns,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'hit_rate': hit_rate,
        'total_trades': total_trades,
        'signals': signals
    }
    
    print(f"\nThreshold: {threshold:.3f}")
    print(f"  Total return: {total_return:.4f}")
    print(f"  Sharpe ratio: {sharpe_ratio:.4f}")
    print(f"  Max drawdown: {max_drawdown:.4f}")
    print(f"  Hit rate: {hit_rate:.3f}")
    print(f"  Total trades: {total_trades}")

# Compare with buy-and-hold
buy_hold_return = np.sum(y_test_btc)
buy_hold_cumulative = np.cumsum(y_test_btc)
buy_hold_sharpe = np.mean(y_test_btc) / np.std(y_test_btc)
buy_hold_drawdown = np.max(np.maximum.accumulate(buy_hold_cumulative) - buy_hold_cumulative)

print(f"\n=== BUY-AND-HOLD BENCHMARK ===")
print(f"Total return: {buy_hold_return:.4f}")
print(f"Sharpe ratio: {buy_hold_sharpe:.4f}")
print(f"Max drawdown: {buy_hold_drawdown:.4f}")

In [ ]:
# Visualize trading strategies
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Cumulative returns comparison
axes[0,0].plot(buy_hold_cumulative.values, label='Buy & Hold', linewidth=2)
for threshold in thresholds[:3]:  # Show top 3 strategies
    axes[0,0].plot(strategy_results[threshold]['cumulative_returns'], 
                   label=f'Strategy (t={threshold})', linewidth=1.5, alpha=0.8)
axes[0,0].set_xlabel('Time')
axes[0,0].set_ylabel('Cumulative Returns')
axes[0,0].set_title('Cumulative Returns Comparison')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Sharpe ratios
sharpe_ratios = [strategy_results[t]['sharpe_ratio'] for t in thresholds]
threshold_labels = [f'{t:.3f}' for t in thresholds]
bars = axes[0,1].bar(threshold_labels, sharpe_ratios, alpha=0.7, color='skyblue')
axes[0,1].axhline(y=buy_hold_sharpe, color='red', linestyle='--', label='Buy & Hold')
axes[0,1].set_xlabel('Threshold')
axes[0,1].set_ylabel('Sharpe Ratio')
axes[0,1].set_title('Sharpe Ratio by Threshold')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Hit rates and total trades
hit_rates = [strategy_results[t]['hit_rate'] for t in thresholds]
total_trades = [strategy_results[t]['total_trades'] for t in thresholds]

ax2 = axes[1,0].twinx()
bars1 = axes[1,0].bar([i-0.2 for i in range(len(thresholds))], hit_rates, 
                      width=0.4, alpha=0.7, color='green', label='Hit Rate')
bars2 = ax2.bar([i+0.2 for i in range(len(thresholds))], total_trades, 
                width=0.4, alpha=0.7, color='orange', label='Total Trades')

axes[1,0].set_xlabel('Threshold')
axes[1,0].set_ylabel('Hit Rate', color='green')
ax2.set_ylabel('Total Trades', color='orange')
axes[1,0].set_title('Hit Rate vs Number of Trades')
axes[1,0].set_xticks(range(len(thresholds)))
axes[1,0].set_xticklabels(threshold_labels)
axes[1,0].grid(True, alpha=0.3)

# Signal distribution for best strategy
best_threshold = max(thresholds, key=lambda x: strategy_results[x]['sharpe_ratio'])
best_signals = strategy_results[best_threshold]['signals']
signal_counts = pd.Series(best_signals).value_counts().sort_index()

signal_labels = ['Short (-1)', 'Hold (0)', 'Long (1)']
signal_colors = ['red', 'gray', 'green']
axes[1,1].pie(signal_counts.values, labels=signal_labels, colors=signal_colors, 
              autopct='%1.1f%%', startangle=90)
axes[1,1].set_title(f'Signal Distribution (Best Strategy: t={best_threshold})')

plt.tight_layout()
plt.show()

print(f"\nBest strategy threshold: {best_threshold} (Sharpe ratio: {strategy_results[best_threshold]['sharpe_ratio']:.4f})")

### Bitcoin Forecasting Insights

**Key Findings:**

1. **Lag Length Impact**: Different lag lengths capture different market dynamics
   - Short lags: Capture immediate momentum
   - Long lags: Capture longer-term patterns

2. **Directional Accuracy**: Often more important than R² for trading
   - Even modest directional accuracy can be profitable
   - R² can be low but strategy still viable

3. **Trading Strategy Performance**: 
   - Threshold selection crucial for risk management
   - Higher thresholds = fewer trades but potentially better quality
   - Transaction costs would significantly impact real performance

**Practical Trading Implications:**
- Linear models provide baseline for more complex strategies
- Risk management (thresholds, position sizing) critical
- Out-of-sample testing essential before live trading
- Consider transaction costs, slippage, and market impact

---

## Exercise 7: Model Comparison

**Task:** Compare linear regression with other simple models.

In [ ]:
# Solution for Exercise 7

print("=== MODEL COMPARISON: LINEAR, RIDGE, AND LASSO ===")

# Test different alpha values for regularization
alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
models_comparison = {}

# Standard Linear Regression (no regularization)
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)

models_comparison['Linear Regression'] = {
    'model': lr_model,
    'alpha': 0,
    'r2_train': r2_score(y_train, lr_model.predict(X_train)),
    'r2_test': r2_score(y_test, lr_pred),
    'rmse_train': np.sqrt(mean_squared_error(y_train, lr_model.predict(X_train))),
    'rmse_test': np.sqrt(mean_squared_error(y_test, lr_pred)),
    'n_features': np.sum(lr_model.coef_[0] != 0),
    'predictions': lr_pred
}

# Ridge Regression with different alphas
best_ridge_alpha = None
best_ridge_score = -np.inf

for alpha in alphas:
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_train, y_train)
    ridge_pred = ridge_model.predict(X_test)
    
    r2_test = r2_score(y_test, ridge_pred)
    
    models_comparison[f'Ridge (α={alpha})'] = {
        'model': ridge_model,
        'alpha': alpha,
        'r2_train': r2_score(y_train, ridge_model.predict(X_train)),
        'r2_test': r2_test,
        'rmse_train': np.sqrt(mean_squared_error(y_train, ridge_model.predict(X_train))),
        'rmse_test': np.sqrt(mean_squared_error(y_test, ridge_pred)),
        'n_features': np.sum(np.abs(ridge_model.coef_[0]) > 1e-6),
        'predictions': ridge_pred
    }
    
    if r2_test > best_ridge_score:
        best_ridge_score = r2_test
        best_ridge_alpha = alpha

# Lasso Regression with different alphas
best_lasso_alpha = None
best_lasso_score = -np.inf

for alpha in alphas:
    lasso_model = Lasso(alpha=alpha, max_iter=2000)
    lasso_model.fit(X_train, y_train.ravel())
    lasso_pred = lasso_model.predict(X_test)
    
    r2_test = r2_score(y_test, lasso_pred)
    
    models_comparison[f'Lasso (α={alpha})'] = {
        'model': lasso_model,
        'alpha': alpha,
        'r2_train': r2_score(y_train, lasso_model.predict(X_train)),
        'r2_test': r2_test,
        'rmse_train': np.sqrt(mean_squared_error(y_train, lasso_model.predict(X_train))),
        'rmse_test': np.sqrt(mean_squared_error(y_test, lasso_pred)),
        'n_features': np.sum(np.abs(lasso_model.coef_) > 1e-6),
        'predictions': lasso_pred
    }
    
    if r2_test > best_lasso_score:
        best_lasso_score = r2_test
        best_lasso_alpha = alpha

# Display results
print("\n=== PERFORMANCE COMPARISON ===")
results_df = pd.DataFrame({
    'Model': list(models_comparison.keys()),
    'Alpha': [models_comparison[m]['alpha'] for m in models_comparison.keys()],
    'R²_Train': [f"{models_comparison[m]['r2_train']:.4f}" for m in models_comparison.keys()],
    'R²_Test': [f"{models_comparison[m]['r2_test']:.4f}" for m in models_comparison.keys()],
    'RMSE_Train': [f"{models_comparison[m]['rmse_train']:.4f}" for m in models_comparison.keys()],
    'RMSE_Test': [f"{models_comparison[m]['rmse_test']:.4f}" for m in models_comparison.keys()],
    'Features': [models_comparison[m]['n_features'] for m in models_comparison.keys()]
})

print(results_df.to_string(index=False))

print(f"\nBest Ridge alpha: {best_ridge_alpha} (R² = {best_ridge_score:.4f})")
print(f"Best Lasso alpha: {best_lasso_alpha} (R² = {best_lasso_score:.4f})")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Extract data for plotting
model_names = list(models_comparison.keys())
r2_train = [models_comparison[m]['r2_train'] for m in model_names]
r2_test = [models_comparison[m]['r2_test'] for m in model_names]
rmse_test = [models_comparison[m]['rmse_test'] for m in model_names]
n_features = [models_comparison[m]['n_features'] for m in model_names]
alphas_plot = [models_comparison[m]['alpha'] for m in model_names]

# R² Train vs Test (overfitting check)
axes[0,0].scatter(r2_train, r2_test, s=60, alpha=0.7)
axes[0,0].plot([0, 1], [0, 1], 'r--', alpha=0.5, label='Perfect generalization')
for i, name in enumerate(model_names):
    if 'Linear' in name or f'α={best_ridge_alpha}' in name or f'α={best_lasso_alpha}' in name:
        axes[0,0].annotate(name.split('(')[0], (r2_train[i], r2_test[i]), 
                          xytext=(5, 5), textcoords='offset points', fontsize=8)
axes[0,0].set_xlabel('R² Train')
axes[0,0].set_ylabel('R² Test')
axes[0,0].set_title('Train vs Test Performance')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Regularization path for Ridge
ridge_alphas = [models_comparison[m]['alpha'] for m in model_names if 'Ridge' in m]
ridge_r2_test = [models_comparison[m]['r2_test'] for m in model_names if 'Ridge' in m]
ridge_features = [models_comparison[m]['n_features'] for m in model_names if 'Ridge' in m]

axes[0,1].semilogx(ridge_alphas, ridge_r2_test, 'bo-', label='Ridge R²')
axes[0,1].axhline(y=models_comparison['Linear Regression']['r2_test'], 
                  color='red', linestyle='--', label='Linear Regression')
axes[0,1].set_xlabel('Alpha (log scale)')
axes[0,1].set_ylabel('R² Test')
axes[0,1].set_title('Ridge Regularization Path')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Regularization path for Lasso
lasso_alphas = [models_comparison[m]['alpha'] for m in model_names if 'Lasso' in m]
lasso_r2_test = [models_comparison[m]['r2_test'] for m in model_names if 'Lasso' in m]
lasso_features = [models_comparison[m]['n_features'] for m in model_names if 'Lasso' in m]

axes[0,2].semilogx(lasso_alphas, lasso_r2_test, 'go-', label='Lasso R²')
axes[0,2].axhline(y=models_comparison['Linear Regression']['r2_test'], 
                  color='red', linestyle='--', label='Linear Regression')
axes[0,2].set_xlabel('Alpha (log scale)')
axes[0,2].set_ylabel('R² Test')
axes[0,2].set_title('Lasso Regularization Path')
axes[0,2].legend()
axes[0,2].grid(True, alpha=0.3)

# Feature selection (Lasso)
axes[1,0].semilogx(lasso_alphas, lasso_features, 'go-', linewidth=2, markersize=8)
axes[1,0].axhline(y=models_comparison['Linear Regression']['n_features'], 
                  color='red', linestyle='--', label='Linear Regression')
axes[1,0].set_xlabel('Alpha (log scale)')
axes[1,0].set_ylabel('Number of Features')
axes[1,0].set_title('Lasso Feature Selection')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Best models comparison
best_models = ['Linear Regression', f'Ridge (α={best_ridge_alpha})', f'Lasso (α={best_lasso_alpha})']
best_r2 = [models_comparison[m]['r2_test'] for m in best_models]
best_rmse = [models_comparison[m]['rmse_test'] for m in best_models]

x_pos = np.arange(len(best_models))
bars = axes[1,1].bar(x_pos, best_r2, alpha=0.7, color=['blue', 'orange', 'green'])
axes[1,1].set_xlabel('Model')
axes[1,1].set_ylabel('R² Test')
axes[1,1].set_title('Best Models Comparison')
axes[1,1].set_xticks(x_pos)
axes[1,1].set_xticklabels([m.split('(')[0] for m in best_models], rotation=45)
axes[1,1].grid(True, alpha=0.3)

# Add value labels on bars
for bar, r2 in zip(bars, best_r2):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 0.001,
                   f'{r2:.4f}', ha='center', va='bottom')

# Coefficient comparison for best models
lr_coef = models_comparison['Linear Regression']['model'].coef_[0]
ridge_coef = models_comparison[f'Ridge (α={best_ridge_alpha})']['model'].coef_[0]
lasso_coef = models_comparison[f'Lasso (α={best_lasso_alpha})']['model'].coef_

# Show top 10 features by absolute coefficient value
top_features_idx = np.argsort(np.abs(lr_coef))[-10:]
feature_names_plot = [X_df.columns[i] for i in top_features_idx]

x_pos_coef = np.arange(len(top_features_idx))
width = 0.25

axes[1,2].bar(x_pos_coef - width, lr_coef[top_features_idx], width, 
              label='Linear', alpha=0.7, color='blue')
axes[1,2].bar(x_pos_coef, ridge_coef[top_features_idx], width, 
              label='Ridge', alpha=0.7, color='orange')
axes[1,2].bar(x_pos_coef + width, lasso_coef[top_features_idx], width, 
              label='Lasso', alpha=0.7, color='green')

axes[1,2].set_xlabel('Features')
axes[1,2].set_ylabel('Coefficient Value')
axes[1,2].set_title('Coefficient Comparison (Top 10 Features)')
axes[1,2].set_xticks(x_pos_coef)
axes[1,2].set_xticklabels([name[:10] + '...' if len(name) > 10 else name 
                          for name in feature_names_plot], rotation=45, ha='right')
axes[1,2].legend()
axes[1,2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Model Comparison Insights

**When to Use Each Model:**

1. **Linear Regression**:
   - When interpretability is crucial
   - Small datasets with few features
   - When you're confident about feature relevance

2. **Ridge Regression**:
   - When you have multicollinearity
   - Want to keep all features but reduce their impact
   - Stable performance across different datasets

3. **Lasso Regression**:
   - When you need automatic feature selection
   - Many irrelevant features in the dataset
   - Want a sparse, interpretable model

**Business Decision Framework:**
- **Interpretability vs Performance**: Simple models for explanation, complex for prediction
- **Feature Importance**: Use Lasso to identify key drivers
- **Stability**: Ridge for consistent performance across time
- **Deployment**: Consider model complexity vs maintenance costs

---

## Exercise 8: Business Impact Analysis

**Task:** Quantify the business value of your duration prediction model.

In [ ]:
# Solution for Exercise 8

print("=== BUSINESS IMPACT ANALYSIS ===")

# Define engagement categories based on call duration
# log(2 minutes) ≈ 0.69, log(5 minutes) ≈ 1.61
def categorize_engagement(log_duration):
    """Categorize engagement based on log duration"""
    if log_duration < 0.69:
        return 'Low'
    elif log_duration < 1.61:
        return 'Medium'
    else:
        return 'High'

# Apply categorization to actual and predicted values
y_test_categories = [categorize_engagement(val) for val in y_test.ravel()]
y_pred_categories = [categorize_engagement(val) for val in y_test_predicted.ravel()]

# Create confusion matrix for engagement levels
from sklearn.metrics import confusion_matrix, classification_report

engagement_labels = ['High', 'Low', 'Medium']  # Alphabetical order for sklearn
conf_matrix = confusion_matrix(y_test_categories, y_pred_categories, labels=engagement_labels)

print("\n=== ENGAGEMENT LEVEL CONFUSION MATRIX ===")
conf_df = pd.DataFrame(conf_matrix, 
                      index=[f'Actual_{label}' for label in engagement_labels],
                      columns=[f'Predicted_{label}' for label in engagement_labels])
print(conf_df)

# Calculate accuracy for each engagement level
print("\n=== CLASSIFICATION REPORT ===")
print(classification_report(y_test_categories, y_pred_categories, labels=engagement_labels))

# Business value calculation
print("\n=== BUSINESS VALUE CALCULATION ===")

# Define business values for each engagement level
engagement_values = {
    'High': 100,    # High engagement calls worth €100 in follow-up value
    'Medium': 30,   # Medium engagement calls worth €30
    'Low': 5        # Low engagement calls worth €5
}

follow_up_cost = 20  # Cost of follow-up call: €20

# Calculate value for different strategies
strategies = {
    'No Model (Random)': {
        'description': 'Follow up with random 50% of customers',
        'selection': np.random.choice([True, False], size=len(y_test_categories), p=[0.5, 0.5])
    },
    'Perfect Model': {
        'description': 'Perfect prediction - follow up only with High engagement',
        'selection': [cat == 'High' for cat in y_test_categories]
    },
    'Our Model (Conservative)': {
        'description': 'Follow up with predicted High engagement only',
        'selection': [cat == 'High' for cat in y_pred_categories]
    },
    'Our Model (Aggressive)': {
        'description': 'Follow up with predicted Medium and High engagement',
        'selection': [cat in ['Medium', 'High'] for cat in y_pred_categories]
    }
}

# Calculate ROI for each strategy
strategy_results = {}

for strategy_name, strategy_info in strategies.items():
    selection = strategy_info['selection']
    
    # Calculate costs and revenues
    n_follow_ups = sum(selection)
    total_cost = n_follow_ups * follow_up_cost
    
    # Calculate revenue from selected customers
    total_revenue = 0
    for i, selected in enumerate(selection):
        if selected:
            actual_engagement = y_test_categories[i]
            total_revenue += engagement_values[actual_engagement]
    
    net_profit = total_revenue - total_cost
    roi = (net_profit / total_cost * 100) if total_cost > 0 else 0
    
    strategy_results[strategy_name] = {
        'n_follow_ups': n_follow_ups,
        'total_cost': total_cost,
        'total_revenue': total_revenue,
        'net_profit': net_profit,
        'roi': roi,
        'profit_per_customer': net_profit / len(y_test_categories)
    }
    
    print(f"\n{strategy_name}:")
    print(f"  Description: {strategy_info['description']}")
    print(f"  Follow-ups: {n_follow_ups}/{len(y_test_categories)} ({n_follow_ups/len(y_test_categories)*100:.1f}%)")
    print(f"  Total cost: €{total_cost:,}")
    print(f"  Total revenue: €{total_revenue:,}")
    print(f"  Net profit: €{net_profit:,}")
    print(f"  ROI: {roi:.1f}%")
    print(f"  Profit per customer: €{net_profit/len(y_test_categories):.2f}")

In [ ]:
# Visualize business impact
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Confusion matrix heatmap
sns.heatmap(conf_df, annot=True, fmt='d', cmap='Blues', ax=axes[0,0])
axes[0,0].set_title('Engagement Level Confusion Matrix')
axes[0,0].set_ylabel('Actual Engagement')
axes[0,0].set_xlabel('Predicted Engagement')

# ROI comparison
strategy_names = list(strategy_results.keys())
rois = [strategy_results[s]['roi'] for s in strategy_names]
colors = ['red', 'green', 'blue', 'orange']

bars = axes[0,1].bar(range(len(strategy_names)), rois, color=colors, alpha=0.7)
axes[0,1].set_xlabel('Strategy')
axes[0,1].set_ylabel('ROI (%)')
axes[0,1].set_title('Return on Investment by Strategy')
axes[0,1].set_xticks(range(len(strategy_names)))
axes[0,1].set_xticklabels([name.replace(' ', '\n') for name in strategy_names], rotation=0)
axes[0,1].grid(True, alpha=0.3)

# Add value labels on bars
for bar, roi in zip(bars, rois):
    height = bar.get_height()
    axes[0,1].text(bar.get_x() + bar.get_width()/2., height + 1,
                   f'{roi:.1f}%', ha='center', va='bottom')

# Net profit comparison
net_profits = [strategy_results[s]['net_profit'] for s in strategy_names]
bars2 = axes[1,0].bar(range(len(strategy_names)), net_profits, color=colors, alpha=0.7)
axes[1,0].set_xlabel('Strategy')
axes[1,0].set_ylabel('Net Profit (€)')
axes[1,0].set_title('Net Profit by Strategy')
axes[1,0].set_xticks(range(len(strategy_names)))
axes[1,0].set_xticklabels([name.replace(' ', '\n') for name in strategy_names], rotation=0)
axes[1,0].grid(True, alpha=0.3)

# Add value labels
for bar, profit in zip(bars2, net_profits):
    height = bar.get_height()
    axes[1,0].text(bar.get_x() + bar.get_width()/2., height + 50,
                   f'€{profit:,.0f}', ha='center', va='bottom')

# Cost vs Revenue breakdown
costs = [strategy_results[s]['total_cost'] for s in strategy_names]
revenues = [strategy_results[s]['total_revenue'] for s in strategy_names]

x_pos = np.arange(len(strategy_names))
width = 0.35

bars3 = axes[1,1].bar(x_pos - width/2, costs, width, label='Costs', color='red', alpha=0.7)
bars4 = axes[1,1].bar(x_pos + width/2, revenues, width, label='Revenues', color='green', alpha=0.7)

axes[1,1].set_xlabel('Strategy')
axes[1,1].set_ylabel('Amount (€)')
axes[1,1].set_title('Cost vs Revenue Breakdown')
axes[1,1].set_xticks(x_pos)
axes[1,1].set_xticklabels([name.replace(' ', '\n') for name in strategy_names], rotation=0)
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Sensitivity analysis
print("\n=== SENSITIVITY ANALYSIS ===")
print("How does changing follow-up cost affect optimal strategy?")

follow_up_costs = [10, 15, 20, 25, 30, 40, 50]
sensitivity_results = {}

for cost in follow_up_costs:
    best_strategy = None
    best_profit = -float('inf')
    
    for strategy_name, strategy_info in strategies.items():
        if strategy_name == 'No Model (Random)':
            continue  # Skip random strategy
            
        selection = strategy_info['selection']
        n_follow_ups = sum(selection)
        total_cost = n_follow_ups * cost
        
        total_revenue = sum(engagement_values[y_test_categories[i]] 
                           for i, selected in enumerate(selection) if selected)
        
        net_profit = total_revenue - total_cost
        
        if net_profit > best_profit:
            best_profit = net_profit
            best_strategy = strategy_name
    
    sensitivity_results[cost] = {
        'best_strategy': best_strategy,
        'best_profit': best_profit
    }
    
    print(f"Follow-up cost €{cost}: Best strategy = {best_strategy} (Profit: €{best_profit:,.0f})")

# Calculate break-even point
print(f"\n=== BREAK-EVEN ANALYSIS ===")
high_engagement_rate = sum(1 for cat in y_test_categories if cat == 'High') / len(y_test_categories)
break_even_cost = engagement_values['High'] * high_engagement_rate
print(f"Break-even follow-up cost for random strategy: €{break_even_cost:.2f}")
print(f"Current follow-up cost: €{follow_up_cost}")
print(f"Margin for error: €{break_even_cost - follow_up_cost:.2f}")

### Business Impact Insights

**Key Business Findings:**

1. **Model Value**: Predictive models can significantly improve ROI vs random selection
2. **Strategy Selection**: Conservative vs aggressive strategies have different risk/reward profiles
3. **Cost Sensitivity**: Follow-up costs critically impact optimal strategy choice
4. **Precision Matters**: Correctly identifying high-value customers drives profitability

**Strategic Recommendations:**

1. **Implement Predictive Targeting**: Even imperfect models beat random selection
2. **Monitor Costs**: Track follow-up costs and adjust strategy accordingly
3. **A/B Testing**: Test different engagement thresholds in practice
4. **Continuous Improvement**: Regularly retrain models with new data

**Risk Considerations:**
- Model predictions may change over time
- Customer behavior may shift
- Competition may affect engagement values
- Economic conditions may impact follow-up success rates

---

## Exercise 9: Advanced Diagnostics

**Task:** Perform advanced model diagnostics to identify potential issues.

In [ ]:
# Solution for Exercise 9

print("=== ADVANCED MODEL DIAGNOSTICS ===")

# Calculate residuals and fitted values
y_fitted = model.predict(X_train)
residuals = y_train.ravel() - y_fitted.ravel()

# 1. Cook's Distance - identifies influential observations
print("\n1. COOK'S DISTANCE ANALYSIS")
print("(Identifying influential observations)")

# Calculate leverage (hat values)
X_train_with_intercept = np.column_stack([np.ones(X_train.shape[0]), X_train])
H = X_train_with_intercept @ np.linalg.inv(X_train_with_intercept.T @ X_train_with_intercept) @ X_train_with_intercept.T
leverage = np.diag(H)

# Calculate Cook's distance
n, p = X_train.shape
mse = np.mean(residuals**2)
cooks_d = (residuals**2 / (p * mse)) * (leverage / (1 - leverage)**2)

# Identify influential points (Cook's D > 4/n)
threshold_cooks = 4 / n
influential_points = np.where(cooks_d > threshold_cooks)[0]

print(f"Cook's Distance threshold (4/n): {threshold_cooks:.6f}")
print(f"Number of influential points: {len(influential_points)}")
print(f"Percentage of influential points: {len(influential_points)/n*100:.2f}%")

if len(influential_points) > 0:
    print(f"Top 5 most influential points (indices): {influential_points[np.argsort(cooks_d[influential_points])[-5:]]}")
    print(f"Their Cook's distances: {cooks_d[influential_points[np.argsort(cooks_d[influential_points])[-5:]]]:.6f}")

# 2. Variance Inflation Factor (VIF) - checks for multicollinearity
print("\n2. MULTICOLLINEARITY ANALYSIS (VIF)")

def calculate_vif(X, feature_names):
    """Calculate VIF for each feature"""
    vif_data = []
    
    for i in range(X.shape[1]):
        # Regress feature i on all other features
        X_others = np.delete(X, i, axis=1)
        
        if X_others.shape[1] > 0:  # Ensure we have other features
            try:
                reg = LinearRegression().fit(X_others, X[:, i])
                r_squared = reg.score(X_others, X[:, i])
                vif = 1 / (1 - r_squared) if r_squared < 0.999 else float('inf')
            except:
                vif = float('inf')
        else:
            vif = 1.0
        
        vif_data.append({
            'Feature': feature_names[i],
            'VIF': vif
        })
    
    return pd.DataFrame(vif_data)

# Calculate VIF for a subset of features (to avoid computational issues)
# Use numerical features only
numerical_feature_indices = [i for i, col in enumerate(X_df.columns) 
                           if any(num_col in col for num_col in ['age', 'previous', 'emp_var_rate', 
                                                                'cons_price_idx', 'cons_conf_idx', 
                                                                'euribor3m', 'nr_employed'])]

if len(numerical_feature_indices) > 1:
    X_numerical_subset = X_train[:, numerical_feature_indices]
    numerical_feature_names = [X_df.columns[i] for i in numerical_feature_indices]
    
    vif_df = calculate_vif(X_numerical_subset, numerical_feature_names)
    vif_df = vif_df.sort_values('VIF', ascending=False)
    
    print("VIF Analysis (Numerical features only):")
    print(vif_df.to_string(index=False))
    
    high_vif = vif_df[vif_df['VIF'] > 10]
    if len(high_vif) > 0:
        print(f"\nFeatures with high multicollinearity (VIF > 10):")
        print(high_vif[['Feature', 'VIF']].to_string(index=False))
    else:
        print("\nNo severe multicollinearity detected (all VIF < 10)")
else:
    print("Insufficient numerical features for VIF analysis")

# 3. Durbin-Watson test for autocorrelation
print("\n3. AUTOCORRELATION ANALYSIS (DURBIN-WATSON TEST)")

def durbin_watson(residuals):
    """Calculate Durbin-Watson statistic"""
    diff = np.diff(residuals)
    return np.sum(diff**2) / np.sum(residuals**2)

dw_stat = durbin_watson(residuals)
print(f"Durbin-Watson statistic: {dw_stat:.4f}")
print(f"Interpretation:")
print(f"  - Close to 2.0: No autocorrelation")
print(f"  - Close to 0.0: Positive autocorrelation")
print(f"  - Close to 4.0: Negative autocorrelation")

if 1.5 <= dw_stat <= 2.5:
    print(f"  - Result: No significant autocorrelation detected")
elif dw_stat < 1.5:
    print(f"  - Result: Positive autocorrelation detected")
else:
    print(f"  - Result: Negative autocorrelation detected")

# 4. Heteroscedasticity tests
print("\n4. HETEROSCEDASTICITY ANALYSIS")

# Breusch-Pagan test (simplified version)
fitted_values = y_fitted.ravel()
squared_residuals = residuals**2

# Regress squared residuals on fitted values
bp_reg = LinearRegression().fit(fitted_values.reshape(-1, 1), squared_residuals)
bp_r_squared = bp_reg.score(fitted_values.reshape(-1, 1), squared_residuals)
bp_statistic = n * bp_r_squared

print(f"Breusch-Pagan test (simplified):")
print(f"  - Test statistic: {bp_statistic:.4f}")
print(f"  - R² of auxiliary regression: {bp_r_squared:.6f}")

# Critical value for chi-square with 1 df at 5% significance
critical_value = 3.841
if bp_statistic > critical_value:
    print(f"  - Result: Heteroscedasticity detected (statistic > {critical_value})")
else:
    print(f"  - Result: No heteroscedasticity detected (statistic < {critical_value})")

In [ ]:
# Visualize advanced diagnostics
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Cook's Distance plot
axes[0,0].stem(range(len(cooks_d)), cooks_d, basefmt=" ")
axes[0,0].axhline(y=threshold_cooks, color='red', linestyle='--', 
                  label=f'Threshold ({threshold_cooks:.6f})')
axes[0,0].set_xlabel('Observation Index')
axes[0,0].set_ylabel("Cook's Distance")
axes[0,0].set_title("Cook's Distance - Influential Points")
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Highlight influential points
if len(influential_points) > 0:
    axes[0,0].scatter(influential_points, cooks_d[influential_points], 
                      color='red', s=50, zorder=5, label='Influential')

# 2. Leverage vs Residuals
axes[0,1].scatter(leverage, residuals, alpha=0.6)
axes[0,1].axhline(y=0, color='red', linestyle='--')
axes[0,1].axvline(x=2*p/n, color='orange', linestyle='--', 
                  label=f'High leverage threshold ({2*p/n:.4f})')
axes[0,1].set_xlabel('Leverage')
axes[0,1].set_ylabel('Residuals')
axes[0,1].set_title('Leverage vs Residuals')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. VIF plot (if available)
if 'vif_df' in locals() and len(vif_df) > 0:
    vif_plot_data = vif_df.head(10)  # Top 10 features
    colors = ['red' if vif > 10 else 'orange' if vif > 5 else 'green' 
              for vif in vif_plot_data['VIF']]
    
    bars = axes[0,2].barh(range(len(vif_plot_data)), vif_plot_data['VIF'], color=colors, alpha=0.7)
    axes[0,2].set_yticks(range(len(vif_plot_data)))
    axes[0,2].set_yticklabels([name[:15] + '...' if len(name) > 15 else name 
                               for name in vif_plot_data['Feature']])
    axes[0,2].axvline(x=5, color='orange', linestyle='--', alpha=0.7, label='Moderate (5)')
    axes[0,2].axvline(x=10, color='red', linestyle='--', alpha=0.7, label='High (10)')
    axes[0,2].set_xlabel('VIF Value')
    axes[0,2].set_title('Variance Inflation Factors')
    axes[0,2].legend()
    axes[0,2].grid(True, alpha=0.3)
else:
    axes[0,2].text(0.5, 0.5, 'VIF analysis\nnot available', 
                   ha='center', va='center', transform=axes[0,2].transAxes)
    axes[0,2].set_title('VIF Analysis')

# 4. Residuals autocorrelation plot
lag_residuals = residuals[:-1]
lead_residuals = residuals[1:]
axes[1,0].scatter(lag_residuals, lead_residuals, alpha=0.6)
axes[1,0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1,0].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[1,0].set_xlabel('Residual(t)')
axes[1,0].set_ylabel('Residual(t+1)')
axes[1,0].set_title(f'Residual Autocorrelation\n(DW = {dw_stat:.3f})')
axes[1,0].grid(True, alpha=0.3)

# 5. Heteroscedasticity plot
axes[1,1].scatter(fitted_values, squared_residuals, alpha=0.6)
# Add trend line
z = np.polyfit(fitted_values, squared_residuals, 1)
p = np.poly1d(z)
axes[1,1].plot(sorted(fitted_values), p(sorted(fitted_values)), "r--", alpha=0.8)
axes[1,1].set_xlabel('Fitted Values')
axes[1,1].set_ylabel('Squared Residuals')
axes[1,1].set_title(f'Heteroscedasticity Test\n(BP = {bp_statistic:.3f})')
axes[1,1].grid(True, alpha=0.3)

# 6. Summary diagnostic plot
diagnostic_summary = {
    'Influential Points': f"{len(influential_points)} ({len(influential_points)/n*100:.1f}%)",
    'High VIF Features': f"{len(high_vif) if 'high_vif' in locals() else 'N/A'}",
    'Autocorrelation': 'Yes' if not (1.5 <= dw_stat <= 2.5) else 'No',
    'Heteroscedasticity': 'Yes' if bp_statistic > critical_value else 'No'
}

y_pos = np.arange(len(diagnostic_summary))
axes[1,2].barh(y_pos, [1]*len(diagnostic_summary), color='lightblue', alpha=0.7)
axes[1,2].set_yticks(y_pos)
axes[1,2].set_yticklabels(list(diagnostic_summary.keys()))
axes[1,2].set_xlim(0, 1)
axes[1,2].set_title('Diagnostic Summary')

# Add text annotations
for i, (key, value) in enumerate(diagnostic_summary.items()):
    axes[1,2].text(0.5, i, value, ha='center', va='center', fontweight='bold')

axes[1,2].set_xticks([])

plt.tight_layout()
plt.show()

In [ ]:
# Recommendations based on diagnostics
print("\n=== DIAGNOSTIC RECOMMENDATIONS ===")

recommendations = []

# Cook's Distance recommendations
if len(influential_points) > n * 0.05:  # More than 5% influential points
    recommendations.append(
        f"⚠️  HIGH INFLUENCE: {len(influential_points)} influential points detected. "
        "Consider investigating these observations for data quality issues or outliers."
    )
elif len(influential_points) > 0:
    recommendations.append(
        f"✓ MODERATE INFLUENCE: {len(influential_points)} influential points detected. "
        "Monitor these observations but no immediate action needed."
    )
else:
    recommendations.append(
        "✓ LOW INFLUENCE: No highly influential points detected. Model is robust to individual observations."
    )

# VIF recommendations
if 'high_vif' in locals() and len(high_vif) > 0:
    recommendations.append(
        f"⚠️  MULTICOLLINEARITY: {len(high_vif)} features with high VIF detected. "
        "Consider removing redundant features or using regularization (Ridge/Lasso)."
    )
else:
    recommendations.append(
        "✓ MULTICOLLINEARITY: No severe multicollinearity detected. Feature set is appropriate."
    )

# Autocorrelation recommendations
if not (1.5 <= dw_stat <= 2.5):
    if dw_stat < 1.5:
        recommendations.append(
            f"⚠️  AUTOCORRELATION: Positive autocorrelation detected (DW = {dw_stat:.3f}). "
            "Consider adding lagged variables or using time series models."
        )
    else:
        recommendations.append(
            f"⚠️  AUTOCORRELATION: Negative autocorrelation detected (DW = {dw_stat:.3f}). "
            "Check for over-differencing or model specification issues."
        )
else:
    recommendations.append(
        f"✓ AUTOCORRELATION: No significant autocorrelation detected (DW = {dw_stat:.3f}). "
        "Residuals appear independent."
    )

# Heteroscedasticity recommendations
if bp_statistic > critical_value:
    recommendations.append(
        f"⚠️  HETEROSCEDASTICITY: Non-constant variance detected (BP = {bp_statistic:.3f}). "
        "Consider using robust standard errors or transforming the target variable."
    )
else:
    recommendations.append(
        f"✓ HOMOSCEDASTICITY: Constant variance assumption appears satisfied (BP = {bp_statistic:.3f})."
    )

# Print recommendations
for i, rec in enumerate(recommendations, 1):
    print(f"\n{i}. {rec}")

# Overall model health score
issues = sum(1 for rec in recommendations if '⚠️' in rec)
total_checks = len(recommendations)
health_score = (total_checks - issues) / total_checks * 100

print(f"\n=== OVERALL MODEL HEALTH SCORE ===")
print(f"Score: {health_score:.1f}% ({total_checks - issues}/{total_checks} checks passed)")

if health_score >= 75:
    print("🟢 GOOD: Model assumptions are largely satisfied. Safe to use for predictions.")
elif health_score >= 50:
    print("🟡 MODERATE: Some assumption violations detected. Use with caution and consider improvements.")
else:
    print("🔴 POOR: Multiple assumption violations detected. Model may be unreliable.")

print(f"\n=== SUGGESTED IMPROVEMENTS ===")
if issues > 0:
    print("Based on the diagnostic results, consider:")
    print("• Data preprocessing: Remove outliers, handle missing values")
    print("• Feature engineering: Transform variables, create interactions")
    print("• Regularization: Use Ridge/Lasso to handle multicollinearity")
    print("• Robust methods: Use robust standard errors for inference")
    print("• Alternative models: Consider non-linear or ensemble methods")
else:
    print("• Model appears healthy! Consider:")
    print("• Cross-validation for performance validation")
    print("• Feature importance analysis for business insights")
    print("• Regular monitoring for model drift in production")

### Advanced Diagnostics Insights

**Why Advanced Diagnostics Matter:**

1. **Model Reliability**: Assumption violations can lead to unreliable predictions
2. **Statistical Inference**: Invalid assumptions affect confidence intervals and p-values
3. **Business Decisions**: Poor model quality can lead to costly business mistakes
4. **Regulatory Compliance**: Some industries require model validation

**Diagnostic Checklist for Production Models:**

✅ **Influential Points**: Check for observations that heavily influence results
✅ **Multicollinearity**: Ensure features provide independent information
✅ **Autocorrelation**: Verify residuals are independent (especially for time series)
✅ **Heteroscedasticity**: Confirm constant variance assumption
✅ **Normality**: Check if residuals follow normal distribution
✅ **Linearity**: Verify linear relationship between features and target

**Business Implementation:**
- Automate diagnostic checks in model pipelines
- Set up alerts for assumption violations
- Document model limitations for stakeholders
- Plan regular model health assessments

---

## Summary and Key Takeaways

Congratulations! You've completed all the linear regression exercises. Here are the key insights:

### 🎯 **Technical Skills Developed**
1. **Model Interpretation**: Understanding coefficients and their business meaning
2. **Assumption Checking**: Validating model assumptions through residual analysis
3. **Feature Engineering**: Creating new features to improve model performance
4. **Model Validation**: Using cross-validation for robust performance estimates
5. **Advanced Diagnostics**: Identifying influential points and assumption violations

### 💼 **Business Applications**
1. **Customer Segmentation**: Using engagement levels for targeted marketing
2. **ROI Analysis**: Quantifying the business value of predictive models
3. **Risk Management**: Understanding model limitations and uncertainties
4. **Strategic Planning**: Using model insights for business decisions

### 🔧 **Best Practices Learned**
1. **Always validate assumptions** before trusting model results
2. **Use cross-validation** for reliable performance estimates
3. **Consider business context** when interpreting model outputs
4. **Balance complexity with interpretability** based on use case
5. **Monitor model health** regularly in production

### 🚀 **Next Steps**
- Apply these techniques to your own datasets
- Explore more advanced regression techniques (polynomial, spline)
- Learn about ensemble methods (Random Forest, Gradient Boosting)
- Study time series analysis for temporal data
- Practice with different business domains and use cases

### 📚 **Additional Resources**
- "An Introduction to Statistical Learning" by James, Witten, Hastie, and Tibshirani
- "The Elements of Statistical Learning" by Hastie, Tibshirani, and Friedman
- Scikit-learn documentation and tutorials
- Kaggle competitions for practical experience

Remember: **The best model is not always the most complex one, but the one that best serves your business objectives while being reliable and interpretable!**